In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.support.select import Select
import time
import re

user = 'eyelash@glittergroupcn.com'
pwd = 'Wangzhen68970747'

chrome_options_headless = webdriver.ChromeOptions()
# chrome_options_headless.add_argument('--headless')
chrome_options_headless.add_argument('--disable-gpu')
chrome_options_headless.add_argument('--disable-extensions')
chrome_options_headless.add_argument('--disable-logging')
chrome_options_headless.add_argument('--ignore-certificate-errors')

In [3]:
browser = webdriver.Chrome(chrome_options=chrome_options_headless)
url = 'https://i.alibaba.com'
browser.get(url)

try:
    WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#alibaba-login-iframe iframe")))

    windows = browser.window_handles
    win = browser.current_window_handle
    for window in windows:
        if window != win:
            browser.switch_to.window(window)
            browser.close()
    browser.switch_to.window(win)

    iframe = browser.find_element_by_css_selector('#alibaba-login-iframe iframe')
    browser.switch_to.frame(iframe)

    input_login = browser.find_element_by_css_selector('input#fm-login-id')
    input_pwd = browser.find_element_by_css_selector('input#fm-login-password')
    input_submit = browser.find_element_by_css_selector('input#fm-login-submit')
    input_login.send_keys(user)
    input_pwd.send_keys(pwd)
    input_submit.click()

    WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'header div[data-role="user"]')))
    if len(browser.find_elements_by_css_selector('header div[data-role="user"]')):
        span = browser.find_element_by_css_selector('header div[data-role="user"] div[data-role="wel"] a span')
        user_name = span.get_attribute('innerHTML')
except TimeoutException:
    pass
# browser.get(url_1)

In [9]:
browser.quit()

In [23]:
ali_id = '60753515570'
pid = '13A03'
path='E:\\Temp\假睫毛\\3D Mink Eyelashes'
isSeried = True
serie='13A-3D貂毛 - series'

attrs = {'alibaba_id': ali_id}
settings = {}

In [11]:
api = 'https://hz-productposting.alibaba.com/product/editing.htm?id='

In [16]:
try:
    browser.get(api+ali_id)
    WebDriverWait(browser, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#editor-container')))
    
    attrs['alibaba_category'] = browser.find_element_by_css_selector('span#cate-path-text-copy').text.strip()
    attrs['name'] = browser.find_element_by_css_selector('input#productName').get_attribute('value').strip()
    keywords = []
    keyword = browser.find_element_by_css_selector('input#productKeyword').get_attribute('value').strip()
    keywords.append(keyword)
    keyword = browser.find_element_by_css_selector('input#keywords2').get_attribute('value').strip()
    keywords.append(keyword)
    keyword = browser.find_element_by_css_selector('input#keywords3').get_attribute('value').strip()
    keywords.append(keyword)
    attrs['keywords'] = keywords
    attrs['type'] = browser.find_element_by_css_selector('#productAttribute>div:first-child>div>input').get_attribute('value').strip()
    attrs['brand'] = browser.find_element_by_css_selector('#productAttribute>div:nth-child(2)>div>input').get_attribute('value').strip()
    
    origin = []
    select = Select(browser.find_elements_by_css_selector('#productAttribute>div:nth-child(3) select')[0])
    origin.append(select.first_selected_option.text)
    
    select = Select(browser.find_elements_by_css_selector('#productAttribute>div:nth-child(3) select')[1])
    origin.append(select.first_selected_option.text)
    attrs['origin'] = origin
    
    select = Select(browser.find_elements_by_css_selector('#productAttribute>div:nth-child(4) select')[0])
    attrs['material'] = select.first_selected_option.text
    
    select = Select(browser.find_elements_by_css_selector('#productAttribute>div:nth-child(5) select')[0])
    attrs['production_method'] = select.first_selected_option.text
    
    custom_attributes = {}
    trs = browser.find_elements_by_css_selector('#attr table tr')
    trs.pop(0)
    for tr in trs:
        key = tr.find_elements_by_css_selector('input')[0].get_attribute('value').strip()
        value = tr.find_elements_by_css_selector('input')[1].get_attribute('value').strip()
        custom_attributes[key] = value
    attrs['custom_attributes'] = custom_attributes
    
    browser.find_element_by_css_selector('#mceu_0 button').click()
    WebDriverWait(browser, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#mceu_46 textarea')))
    time.sleep(1)
    settings['template'] = browser.find_element_by_css_selector('#mceu_46 textarea').get_attribute('value').strip()
    browser.find_element_by_css_selector('#mceu_0 button').click()
    WebDriverWait(browser, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#mceu_46 iframe')))
    time.sleep(1)
    
    css_selector = '#tradInfoblock>div:nth-child(3) .price-scroll-selector input[value="uniform"]'
    element = browser.find_element_by_css_selector(css_selector)
    attrs['isTieredPricing'] = element.is_selected()
    if(attrs['isTieredPricing']):
        attrs['sales_unit'] = browser.find_element_by_css_selector('#tradeInfoQuantityUnit').text.strip()
        attrs['currency'] = 'USD'
        tieredPricing = []
        trs = browser.find_elements_by_css_selector('#priceSetter .next-table-body tbody tr')
        for tr in trs:
            volume = tr.find_elements_by_css_selector('input')[0].get_attribute('value').strip()
            price = tr.find_elements_by_css_selector('input')[1].get_attribute('value').strip()
            tieredPricing.append([volume, price])
        attrs['tieredPricing'] = tieredPricing
    else:
        attrs['currency'] = browser.find_element_by_css_selector('#moneyTypeSelect span').text.strip()
        price_range = []
        price_range.append(browser.find_element_by_css_selector('#priceRangeMin').get_attribute('value').strip())
        price_range.append(browser.find_element_by_css_selector('#priceRangeMax').get_attribute('value').strip())
        attrs['price_range'] = price_range
        attrs['sales_unit'] = browser.find_element_by_css_selector('#priceUnitSelect span').text.strip()
        attrs['MOQ'] = browser.find_element_by_css_selector('#minOrderQuantity').get_attribute('value').strip()
        attrs['additional_trading_infomation'] = browser.find_element_by_css_selector('#minOrderOther').get_attribute('value').strip()
    payment_methods = {}
    checkboxes = browser.find_elements_by_css_selector('.trade-info-payment-group input[type="checkbox"]')
    for cb in checkboxes:
        value = cb.get_attribute('value')
        checked = cb.is_selected()
        if(value=='Others'):
            continue
        payment_methods[value] = checked
    attrs['payment_methods'] = payment_methods
    if browser.find_element_by_css_selector('#paymentMethodOther').is_selected():
        attrs['other_payment_method'] = browser.find_element_by_css_selector('#paymentMethodOtherDesc').get_attribute('value')
        
    attrs['delivery_time'] = browser.find_element_by_css_selector('#consignmentTerm').get_attribute('value')
    attrs['port'] = browser.find_element_by_css_selector('#port').get_attribute('value')
    supply_ability = []
    supply_ability.append(browser.find_element_by_css_selector('#supplyQuantity').get_attribute('value'))
    select = Select(browser.find_element_by_css_selector('#supplyUnit'))
    supply_ability.append(select.first_selected_option.text)
    select = Select(browser.find_element_by_css_selector('#supplyPeriod'))
    supply_ability.append(select.first_selected_option.text)
    attrs['supply_ability'] = supply_ability
    attrs['additional_delivery_information'] = browser.find_element_by_css_selector('#supplyOther').get_attribute('value')
    attrs['packaging'] = browser.find_element_by_css_selector('#packagingDesc').get_attribute('value')
    
except TimeoutException:
    attrs = None
    settings = None
except NoSuchElementException:
    attrs = None
    settings = None

if( not attrs or not settings):
    # errors:
else:

------------------


{'L/C': True,
 'D/A': False,
 'D/P': False,
 'T/T': True,
 'Western Union': True,
 'MoneyGram': False}

In [136]:
packagingDesc

'Pair/Pairs'

In [167]:
browser.find_element_by_css_selector('#supplyOther').get_attribute('value')

''

In [166]:
select = Select(browser.find_element_by_css_selector('#supplyPeriod'))
select.first_selected_option.text

'Month'

In [117]:

print('++++++++++++++++++')

++++++++++++++++++


In [116]:
browser.find_elements_by_css_selector('textarea')

[<selenium.webdriver.remote.webelement.WebElement (session="2c52ff7728332ee2985eefdd937319e5", element="0.005622564638980032-353")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2c52ff7728332ee2985eefdd937319e5", element="0.005622564638980032-349")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2c52ff7728332ee2985eefdd937319e5", element="0.005622564638980032-350")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2c52ff7728332ee2985eefdd937319e5", element="0.005622564638980032-354")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2c52ff7728332ee2985eefdd937319e5", element="0.005622564638980032-355")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2c52ff7728332ee2985eefdd937319e5", element="0.005622564638980032-356")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2c52ff7728332ee2985eefdd937319e5", element="0.005622564638980032-357")>]

In [28]:
mceu_46

In [113]:
import time
from selenium.webdriver.support.select import Select

In [96]:
browser.switch_to.default_content()

In [168]:
bds = None
bds.split("")

AttributeError: 'NoneType' object has no attribute 'split'

In [169]:
browser.quit()